In [6]:
import os
import sys
import numpy as np
import pandas as pd
from sklearn.externals import joblib

from sklearn.utils import shuffle
sys.path.append( os.getcwd()+'/modules' )
import aux_functions

In [2]:
#Loading datasets
path = os.getcwd() 
os.chdir( "/home/toful/Documents/DataSets/cresci-2017.csv/datasets_full.csv/" )

real = pd.read_csv( 'genuine_accounts.csv/genuine_accounts.csv/users.csv' )
real = real.fillna( '' )
real['knownbot'] = 0

fakeFollowers = pd.read_csv('fake_followers.csv/fake_followers.csv/users.csv', na_filter=False)
fakeFollowers = fakeFollowers.fillna( '' )
fakeFollowers['knownbot'] = 1

df = pd.concat( [ real, fakeFollowers ], sort=False )
df = shuffle( df )
#print df.head()
#print df.info()

os.chdir( path )


In [10]:
#Building the working dataset
score = pd.DataFrame()
score['id'] = df['id']

score['lang-en'] = df.apply( lambda row: aux_functions.language (row), axis=1 )
score['profile_pic'] = df.apply( lambda row: aux_functions.profile_image (row), axis=1 ) #check this feature
score['def_profile_pic'] = df.apply( lambda row: aux_functions.def_profile_image (row), axis=1 )
score['has_screen_name'] = df.apply( lambda row: aux_functions.screen_name (row), axis=1 )
score['30followers'] = df.apply( lambda row: aux_functions.min_followers (row, 30), axis=1 )
score['1000friends'] = df.apply( lambda row: aux_functions.min_friends (row, 1000), axis=1 )
score['twice_num_followers'] = df.apply( lambda row: aux_functions.ratio_followers (row), axis=1 )
score['fifty_FriendsFollowersRatio'] = df.apply( lambda row: aux_functions.ratio_followers2 (row, 50), axis=1 )
score['hundred_FriendsFollowersRatio'] = df.apply( lambda row: aux_functions.ratio_followers2 (row, 100), axis=1 )
score['geoloc'] = df.apply( lambda row: aux_functions.location (row), axis=1 )
score['banner_link'] = df.apply( lambda row: aux_functions.profile_banner (row), axis=1 )
score['50tweets'] = df.apply( lambda row: aux_functions.tweets_written (row, 50), axis=1 )
score['20statuses'] = df.apply( lambda row: aux_functions.min_statuses (row, 20), axis=1 )
score['NeverTweeted'] = df.apply( lambda row: aux_functions.never_tweeted (row ), axis=1 )
score['has_description'] = df.apply( lambda row: aux_functions.description (row), axis=1 )
score['knownbot'] = df.apply( lambda row: aux_functions.knownbot (row), axis=1 )

print score.shape
print score.describe()
print score.columns.values
#score

(6825, 17)
                 id      lang-en  profile_pic  def_profile_pic  \
count  6.825000e+03  6825.000000       6825.0      6825.000000   
mean   8.499737e+08     0.898022          0.0         0.002784   
std    7.662248e+08     0.302642          0.0         0.052693   
min    6.780330e+05     0.000000          0.0         0.000000   
25%    2.597067e+08     1.000000          0.0         0.000000   
50%    6.168972e+08     1.000000          0.0         0.000000   
75%    1.174963e+09     1.000000          0.0         0.000000   
max    3.164942e+09     1.000000          0.0         1.000000   

       has_screen_name  30followers  1000friends  twice_num_followers  \
count      6825.000000  6825.000000  6825.000000          6825.000000   
mean          0.000147     0.492601     0.076337             0.588425   
std           0.012105     0.499982     0.265556             0.492155   
min           0.000000     0.000000     0.000000             0.000000   
25%           0.000000     0.

# Data Analisis

In [4]:
from sklearn.model_selection import ShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics as mt

In [5]:
#Preparing the X and Y values for the machine learning analisis
if 'knownbot' in score:
    y = score['knownbot'].values # get the labels we want
    del score['knownbot'] # get rid of the class label
    X = score.values # use everything else to predict

In [12]:
num_cv_iterations = 3
num_instances = len(y)
cv_object = ShuffleSplit( n_splits=num_cv_iterations, test_size = 0.2 )

# first we create a reusable logisitic regression and random forest objects
lr_clf = LogisticRegression( penalty='l2', C=1.0, class_weight=None ) # get object
rf_clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=0)

iter_num=0
# the indices are the rows used for training and testing in each iteration
for train_indices, test_indices in cv_object.split( X, y ):
    X_train = X[train_indices]
    y_train = y[train_indices]
    X_test = X[test_indices]
    y_test = y[test_indices]
    
    # train the reusable logisitc regression model on the training data
    #lr_clf.fit( X_train, y_train ) # train object
    #y_pred = lr_clf.predict( X_test ) # get test set precitions
   
    rf_clf.fit( X_train, y_train ) # train object
    y_pred = rf_clf.predict( X_test ) # get test set precitions
    acc = mt.accuracy_score( y_test, y_pred )
    conf = mt.confusion_matrix( y_test, y_pred )
    # now let's get the accuracy and confusion matrix for this iterations of training/testing
    print "\n====Iteration",iter_num," ===="
    print "RANDOM FOREST"
    print "accuracy: ", acc
    print "confusion matrix\n", conf
    tp = float( conf[1][1] )
    fp = float( conf[0][1] )
    fn = float( conf[1][0] )
    precission = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2*precission*recall/(precission+recall)
    print "F1: ", f1, "\tPrecission: ", precission, "\tRecall: ", recall
    iter_num+=1

# save the model to disk
if not os.path.exists( 'models' ):
    os.makedirs( 'models' ) 
joblib.dump( rf_clf, 'models/randomForest_model.sav')
    


====Iteration 0  ====
RANDOM FOREST
accuracy:  0.9838827838827838
confusion matrix
[[680   5]
 [ 17 663]]
F1:  0.983679525223 	Precission:  0.99251497006 	Recall:  0.975

====Iteration 1  ====
RANDOM FOREST
accuracy:  0.9824175824175824
confusion matrix
[[695  13]
 [ 11 646]]
F1:  0.981762917933 	Precission:  0.980273141123 	Recall:  0.983257229833

====Iteration 2  ====
RANDOM FOREST
accuracy:  0.9868131868131869
confusion matrix
[[708   7]
 [ 11 639]]
F1:  0.986111111111 	Precission:  0.989164086687 	Recall:  0.983076923077


['models/randomForest_model.sav']

In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

In [ ]:
#Setting up the model layers
#https://www.tensorflow.org/api_docs/python/tf/keras/layers
#https://www.tensorflow.org/tutorials/keras/basic_classification
model = keras.Sequential( [ keras.layers.Dense( 32, input_shape=( len(X[0]), ) ),
                            keras.layers.Dense( 40, activation = tf.nn.sigmoid ),
                            keras.layers.Dense( 10, activation = tf.nn.sigmoid ),
                            keras.layers.Dense( 2, activation = tf.nn.softmax ) ] )
#model = keras.Sequential( )
#model.add( keras.layers.Dense( 32, input_shape = ( len(X[0]), ) ) )
#model.add( keras.layers.Dense( 2 ) )

#the optimazer has to minimize the loss function

#Compiling the model
  #Loss function — This measures how accurate the model is during training. We want to minimize 
                   #this function to "steer" the model in the right direction.
  #Optimizer — This is how the model is updated based on the data it sees and its loss function.
  #Metrics —  Used to monitor the training and testing steps. The following example uses accuracy,
              #the fraction of the images that are correctly classified.
model.compile( optimizer = 'adam', 
               loss = 'mean_squared_error',
               metrics = ['accuracy'] )

num_cv_iterations = 3
cv_object = ShuffleSplit( n_splits=num_cv_iterations, test_size = 0.2 )
iter_num = 1

for train_indices, test_indices in cv_object.split( X, y ):
    X_train = X[train_indices]
    y_train = y[train_indices]
    X_test = X[test_indices]
    y_test = y[test_indices]
    
    print "\n\n====Iteration",iter_num," ===="
    
    model.fit( X_train, y_train, epochs=100 )
    test_loss, test_acc = model.evaluate( X_test, y_test )
    y_pred = model.predict( X_test )
    aux=[]
    for i in range( len( y_pred ) ):
        aux += [ np.argmax( y_pred[i] ) ]
    conf = mt.confusion_matrix( y_test, aux )
    
    print '\nMODEL RESULTS\n\tTest accuracy: ', test_acc, 'Test loss: ', test_loss
    print "PREDICTION RESULTS"
    print "confusion matrix\n", conf
    tp = float( conf[1][1] )
    fp = float( conf[0][1] )
    fn = float( conf[1][0] )
    try:  
        precission = tp / (tp + fp)
        recall = tp / (tp + fn)
        f1 = 2*precission*recall/(precission+recall)
    except ArithmeticError:  
        print ("Arithmetic exception raised." )
    else: 
        print "\tF1: ", f1, "\tPrecission: ", precission, "\tRecall: ", recall
    iter_num+=1

#print y_pred